In [ ]:
import os
import csv
import json
import logging
import math
from typing import List, Dict, Tuple, Callable, Iterable, Optional, Any

import torch
import torch.nn as nn
import pandas as pd
import pickle
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_ = 'drive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator, SentenceEvaluator
from sentence_transformers.util import batch_to_device
from torch import Tensor
from torch.utils.data import DataLoader
from transformers import MT5EncoderModel, AutoConfig,T5Tokenizer
from transformers import pipeline, AutoTokenizer

In [ ]:
train_df=pd.read_csv(folder_ + 'Data/Train.csv')
#test_df=pd.read_csv('C:/Users/leose/second_semester/NLP/malawi_news_classification/data/test.csv')

In [ ]:
train_df['Text']=train_df['Text'].tolist()
#test_df['Text']=test_df['Text'].tolist()
train_batch_sentences=[]
#test_batch_sentences=[]

#for frame in train_df['Text']:
#    train_batch_sentences.append(frame)
#for sentence in test_df['Text']:
#    test_batch_sentences.append(frame)

In [ ]:
model = MT5EncoderModel.from_pretrained("google/mt5-small")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
#article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
#for sentence
#input_ids = tokenizer(train_batch_sentences[0], return_tensors="pt",truncation=True,max_length=512).input_ids
#outputs = model(input_ids)
#hidden_state = outputs.last_hidden_state

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5EncoderModel: ['decoder.block.3.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.4.layer.0.SelfAttention.q.weight', 'decoder.block.7.layer.1.EncDecAttention.k.weight', 'decoder.block.4.layer.2.DenseReluDense.wo.weight', 'decoder.block.2.layer.2.DenseReluDense.wo.weight', 'decoder.block.1.layer.1.layer_norm.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.7.layer.2.DenseReluDense.wo.weight', 'decoder.block.7.layer.0.SelfAttention.k.weight', 'decoder.block.5.layer.2.layer_norm.weight', 'decoder.block.2.layer.1.EncDecAttention.q.weight', 'decoder.block.2.layer.1.EncDecAttention.v.weight', 'decoder.block.2.layer.0.SelfAttention.q.weight', 'decoder.block.6.layer.0.layer_norm.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.4.layer.1.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.1.layer.0.Sel

In [ ]:
input_ids = []
for frame in train_df['Text'][:400]:
    input_ids.append(tokenizer(frame, return_tensors="pt",truncation=True,max_length=64).input_ids)

In [ ]:
len(input_ids)

400

In [ ]:
outputs = []
for input_ in input_ids:
  outputs.append(model(input_))

In [ ]:
#outputs = model(input_ids)

In [ ]:
hidden_state = []
for output_ in outputs:
  hidden_state.append(output_.last_hidden_state)

In [ ]:
hidden_state[0].shape

torch.Size([1, 64, 512])

In [ ]:
hidden_state[0][0].shape

torch.Size([64, 512])

In [ ]:
hidden_state[0][0][0]

tensor([ 7.4249e-02, -2.2944e-01, -1.2161e-01, -1.1340e-01, -2.7140e-01,
         1.3483e-01,  6.2602e-01,  3.5955e-02,  8.0742e-03,  1.2149e-03,
         3.4012e-01, -6.8906e-01, -2.0506e-02,  8.0262e-02, -1.4345e-01,
         4.5232e-01, -2.0572e-01, -5.7143e-02,  9.4847e-02,  1.0415e-02,
        -4.0238e-01,  1.1234e-01, -7.5072e-01,  2.2831e-01,  1.0428e-01,
        -2.3438e-01, -2.3618e-01, -2.1552e-01,  8.2930e-02,  4.2428e-02,
        -9.9660e-02,  2.4933e-02, -3.5620e-03,  2.8719e-01,  4.6472e-01,
         5.7856e-02,  1.0510e-01,  2.4711e-01,  5.2340e-01, -2.3537e-02,
        -1.5441e-02,  1.1511e-01,  2.9481e-01, -5.5164e-01,  2.1864e-01,
        -4.0927e-01, -3.9705e-01, -1.5833e-01, -1.1707e-01,  1.0994e-02,
        -2.0989e-01, -2.0662e-01, -2.2973e-01, -2.8794e-01, -5.6752e-01,
         1.2604e-02, -5.2044e-02, -1.1523e-01, -1.3299e-01,  1.1255e-01,
        -2.1574e-01, -2.2517e-01,  3.3094e-01,  4.5194e-01, -9.0863e-02,
         4.0706e-01,  4.4283e-02,  2.9414e-01,  6.5

In [ ]:
train_df.shape

(1436, 3)

In [ ]:
len(train_batch_sentences)

1436

In [ ]:
train_representations=tokenizer(train_batch_sentences,padding=True,truncation=True)
#test_representations=tokenizer(test_batch_sentences,padding=True,truncation=True)

In [ ]:
np.unique(train_df['Label']).shape[0]

20

In [ ]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [ ]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [ ]:
print(Y_train.shape)
print(Y_train[0])

(1436, 20)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(64,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(20))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#make one, takes in, double that amount, english and chichewa

Building model...


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 512)               33280     
                                                                 
 activation_1 (Activation)   (None, 512)               0         
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 20)                10260     
                                                                 
 activation_2 (Activation)   (None, 20)                0         
                                                                 
Total params: 43,540
Trainable params: 43,540
Non-trainable params: 0
_________________________________________________________________


In [ ]:
len(hidden_state)

400

In [ ]:
hidden_state[0].shape

torch.Size([1, 64, 512])

In [ ]:
a = []
for i in range(len(hidden_state)):
  a.append(hidden_state[i])

b = torch.Tensor(len(hidden_state), 1,64,512)
torch.cat(a,out=b)

RuntimeError: ignored

In [ ]:
for i in range(len(hidden_state)):
  b[i] = hidden_state[i]

In [ ]:
b[0]

tensor([[[ 0.0742, -0.2294, -0.1216,  ...,  0.3356,  0.5099, -0.0483],
         [ 0.1420, -0.2561, -0.0410,  ...,  0.6036,  0.2028, -0.2840],
         [ 0.1522, -0.3163, -0.6257,  ...,  0.2101,  0.3074, -0.1771],
         ...,
         [ 0.0079, -0.1694, -0.0251,  ..., -0.2738, -0.2077,  0.0914],
         [ 0.3339,  0.1480, -0.2112,  ..., -0.1909, -0.2393,  0.1825],
         [-0.0358, -0.0088,  0.0080,  ..., -0.0185,  0.0023,  0.0194]]],
       grad_fn=<SelectBackward0>)

In [ ]:
hidden_state[0]

tensor([[[ 0.0742, -0.2294, -0.1216,  ...,  0.3356,  0.5099, -0.0483],
         [ 0.1420, -0.2561, -0.0410,  ...,  0.6036,  0.2028, -0.2840],
         [ 0.1522, -0.3163, -0.6257,  ...,  0.2101,  0.3074, -0.1771],
         ...,
         [ 0.0079, -0.1694, -0.0251,  ..., -0.2738, -0.2077,  0.0914],
         [ 0.3339,  0.1480, -0.2112,  ..., -0.1909, -0.2393,  0.1825],
         [-0.0358, -0.0088,  0.0080,  ..., -0.0185,  0.0023,  0.0194]]],
       grad_fn=<MulBackward0>)

In [ ]:
type(b)

torch.Tensor

In [ ]:
b.shape

torch.Size([400, 1, 64, 512])

In [ ]:
result = model.fit(b, Y_train[:400],
                    epochs=2, batch_size=32,
                    verbose=1)

TypeError: ignored

https://austingwalters.com/classify-sentences-via-a-multilayer-perceptron-mlp/
https://github.com/holbertonschool/deep-learning/blob/master/Class%20%233/Text%20Classification%20MLP%20Reuters%20Dataset.ipynb
https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2


Hidden state (embeddings):
https://github.com/huggingface/transformers/issues/1950